<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/final_project12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
uploaded=files.upload()

In [ ]:
서울_양파_가격=pd.read_excel('/content/서울전역시장마트데이터(구분포함,23월포함).xlsx')
양파_특_가락시장_data=pd.read_excel('/content/양파_특.xlsx')
서울날씨=pd.read_excel('/content/seoul_weather.xlsx')
공휴일=pd.read_excel('/content/holiday.xlsx')

In [ ]:
서울_양파_가격.head()

In [ ]:
서울날씨.rename(columns = {'일시':'일자'}, inplace = True)
서울날씨.head()

In [ ]:
공휴일.head()

In [ ]:
서울_양파_가격=서울_양파_가격[(서울_양파_가격['품목']=='양파')&(서울_양파_가격['구분']=='마트')]

In [ ]:
서울_양파_가격.info()

In [ ]:
서울_양파_가격=서울_양파_가격.groupby('조사일').mean()

In [ ]:
서울_양파_가격.head()

In [ ]:
서울_양파_가격.reset_index('조사일',inplace = True)

In [ ]:
서울_양파_가격.rename(columns = {'조사일':'일자'}, inplace = True)

In [ ]:
서울_양파_가격.head()

In [ ]:
양파_특_가락시장_data.rename(columns = {'평균가격':'경락가평균가격'}, inplace = True)
양파_특_가락시장_data.head()

In [ ]:
양파_통합 =pd.merge(서울_양파_가격, 양파_특_가락시장_data, on='일자')

In [ ]:
양파_통합=pd.merge(양파_통합,공휴일,on='일자')

In [ ]:
양파_통합=pd.merge(양파_통합,서울날씨,on='일자')

In [ ]:
양파_통합.to_excel('양파_마트통합.xlsx')

In [ ]:
양파_통합=양파_통합[['가격','반입량','경락가평균가격','유가 전국평균가격','유무','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)']]

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
양파_통합.data=양파_통합[['반입량','경락가평균가격','유가 전국평균가격','최저기온(°C)','최고기온(°C)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 5cm 지중온도(°C)']]

In [ ]:
#양파_통합.data=양파_통합[['반입량','경락가평균가격','유가 전국평균가격','평균 풍속(m/s)']]

In [ ]:
# 했을떄 0.92
#지면온도 강수량 유무 빼고 0.93


In [ ]:
양파_통합.target=양파_통합[['가격']]

In [ ]:
x,y=양파_통합.data,양파_통합.target

In [ ]:
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
xgbr.fit(xtrain, ytrain)

In [ ]:
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)

In [ ]:
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)

In [ ]:
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_importance(xgbr)
plt.show()

In [ ]:
#'일강수량(mm)','평균 상대습도(%)' 중요도 낮은거 두개빼니까 60%까지 올라감